In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
df = pd.read_csv('postcode-dataout.txt', header=None, names=['suburb', 'state', 'postcode'], dtype={'suburb': str, 'state': str, 'postcode': str})
gdf = gpd.read_file('suburb-10-vic.geojson')

In [3]:
# Find duplicates in suburb and state
dp = df[df.duplicated(['suburb', 'state'], keep=False)]
dp = dp[dp['state'] == 'VIC']
du = df[~df.duplicated(['suburb', 'state'], keep=False)]
du = du[du['state'] == 'VIC']

In [4]:
gdp = gdf[gdf.duplicated(['vic_loca_2'], keep=False)].sort_values('vic_loca_2')
gdu = gdf[~gdf.duplicated(['vic_loca_2'], keep=False)].sort_values('vic_loca_2')

In [5]:
dgu = pd.merge(gdu, du, left_on=['vic_loca_2'], right_on=['suburb'], how='left',)

In [6]:
gdx = dgu[dgu['suburb'].isna()]

In [7]:
dgp = pd.merge(gdp, dp, left_on=['vic_loca_2', 'vic_loca_4'], right_on=['suburb', 'postcode'], how='left')

In [8]:
dgp[dgp['suburb'].isna()]

,id,lc_ply_pid,dt_create,dt_retire,loc_pid,vic_locali,vic_loca_1,vic_loca_2,vic_loca_3,vic_loca_4,vic_loca_5,vic_loca_6,vic_loca_7,geometry,suburb,state,postcode


In [9]:
# dp.groupby(['suburb', 'state'])['postcode'].apply(lambda x: ', '.join(x)).reset_index()
# Get a list of postcode for each suburb
dx = dp.groupby(['suburb', 'state'])['postcode'].apply(lambda x: ', '.join(x)).reset_index()

In [10]:
dgx = pd.merge(gdx, dx, left_on=['vic_loca_2'], right_on=['suburb'], how='left')

In [11]:
dgx[dgx['postcode_y'].isna()]

,id,lc_ply_pid,dt_create,dt_retire,loc_pid,vic_locali,vic_loca_1,vic_loca_2,vic_loca_3,vic_loca_4,vic_loca_5,vic_loca_6,vic_loca_7,geometry,suburb_x,state_x,postcode_x,suburb_y,state_y,postcode_y
0,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.2755,10641,2017-06-02,None,VIC3010,2017-08-09,None,AINTREE,None,None,G,None,2,"POLYGON ((144.68012 -37.73797, 144.67204 -37.7...",NaN,NaN,NaN,NaN,NaN,NaN
4,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.2965,10642,2017-06-02,None,VIC3011,2017-08-09,None,BONNIE BROOK,None,None,G,None,2,"POLYGON ((144.69246 -37.71659, 144.68530 -37.7...",NaN,NaN,NaN,NaN,NaN,NaN
6,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.2888,10152,2015-09-03,None,VIC3006,2015-11-09,None,CREEK VIEW,None,None,G,None,2,"POLYGON ((144.58958 -36.58349, 144.60227 -36.5...",NaN,NaN,NaN,NaN,NaN,NaN
19,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.2725,10650,2017-06-02,None,VIC3019,2017-08-09,None,THORNHILL PARK,None,None,G,None,2,"POLYGON ((144.63185 -37.74547, 144.63063 -37.7...",NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Custom postcode for suburb
# AINTREE 3336
# BONNIE BROOK 3335
# CREEK VIEW 3551
# THORNHILL PARK 3335
df_add = pd.DataFrame({'suburb': ['AINTREE', 'BONNIE BROOK', 'CREEK VIEW', 'THORNHILL PARK'], 'state': ['VIC', 'VIC', 'VIC', 'VIC'], 'postcode': ['3336', '3335', '3551', '3335']})
df = pd.concat([df, df_add])

In [13]:
dfx = df.dropna(subset=['postcode']).groupby(['suburb', 'state'])['postcode'].apply(lambda x: ', '.join(x)).reset_index()
dfx = dfx[dfx['state'] == 'VIC']
gdfx = pd.merge(gdf, dfx, left_on='vic_loca_2', right_on='suburb', how='left')
gdfx['vic_loca_4'].fillna(gdfx['postcode'], inplace=True)


In [14]:
gdfx['loc_pid'].is_unique, gdfx['lc_ply_pid'].is_unique

(True, True)

In [15]:
gdfx['name'] = gdfx['vic_loca_2']
gdfx['postcode'] = gdfx['vic_loca_4']
gdfx['id'] = gdfx['loc_pid']

In [16]:
gdfx = gdfx[['id', 'name', 'postcode', 'geometry']]

In [17]:
gdfx.to_file('suburbs-vic.geojson', driver='GeoJSON')